In [1]:
import cobra

model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"
model = cobra.io.read_sbml_model(model_path)

biomass_rxns = [r for r in model.reactions if "biomass" in r.id.lower() or "biomass" in (r.name or "").lower()]
print("Biomass reactions found:", [r.id for r in biomass_rxns])

No objective coefficients in model. Unclear what should be optimized


Biomass reactions found: ['BIOMASS_Ec_SynAuto_1', 'BIOMASS_Ec_SynMixo_1', 'BIOMASS_Ec_SynHetero_1']


In [2]:
if biomass_rxns:
    model.objective = biomass_rxns[0]
    sol = model.optimize()
    print("WT Growth:", sol.objective_value)


WT Growth: 0.07653046659469502


In [3]:
for rxn in model.exchanges:
    rxn.lower_bound = -10  # allow uptake
print("WT Growth after opening exchanges:", model.optimize().objective_value)

WT Growth after opening exchanges: 1.6579776821461234


In [6]:
import pandas as pd

def simulate_knockout(model, gene_id):
    wt = model.optimize().objective_value
    with model:
        try:
            gene = model.genes.get_by_id(gene_id)
            for rxn in gene.reactions:
                rxn.knock_out()
            ko = model.optimize().objective_value
            pct = ((ko - wt) / wt * 100) if wt else None
            return wt, ko, pct
        except KeyError:
            return wt, None, None

In [7]:
rows = []
for g in model.genes:
    wt, ko, pct = simulate_knockout(model, g.id)
    rows.append({"gene": g.id, "WT_growth": wt, "KO_growth": ko, "Pct_change": pct})

df = pd.DataFrame(rows)
df.to_csv("results/knockout_results.csv", index=False)
print("Saved knockout_results.csv")

OSError: Cannot save file into a non-existent directory: 'results'

In [8]:
import cobra
import pandas as pd
from cobra import Reaction

# -----------------------------
# 1. Load the GEM
# -----------------------------
model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"
model = cobra.io.read_sbml_model(model_path)

print(f"Model loaded with {len(model.reactions)} reactions, {len(model.metabolites)} metabolites, {len(model.genes)} genes")

# -----------------------------
# 2. Open all exchange reactions
# -----------------------------
for rxn in model.exchanges:
    rxn.lower_bound = -10  # allow uptake at reasonable rate

print("All exchanges opened.")

# -----------------------------
# 3. Check for biomass reaction or add one
# -----------------------------
if model.objective is None or len(model.objective.keys()) == 0:
    print("No biomass reaction found. Adding test biomass...")
    rxn = Reaction("BIOMASS_test")
    
    # Select simple metabolites
    met_ids = {"atp_c","adp_c","pi_c","h2o_c","h_c"}
    mets = {m.id: m for m in model.metabolites if m.id in met_ids}
    
    # Add stoichiometry for ATP hydrolysis as dummy biomass
    rxn.add_metabolites({
        mets["atp_c"]: -1,
        mets["h2o_c"]: -1,
        mets["adp_c"]: 1,
        mets["pi_c"]: 1,
        mets["h_c"]: 1
    })
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    model.add_reactions([rxn])
    model.objective = rxn
    print("Test biomass reaction added.")
else:
    print("Biomass reaction found:", list(model.objective.keys())[0])

# -----------------------------
# 4. Check wild-type growth
# -----------------------------
wt_growth = model.optimize().objective_value
print(f"Wild-type growth rate: {wt_growth}")

if wt_growth == 0:
    print("⚠️ Warning: WT growth is zero. Knockouts will not show changes.")

# -----------------------------
# 5. Function for single-gene knockout
# -----------------------------
def simulate_knockout(model, gene_id):
    with model:
        cobra.manipulation.delete_model_genes(model, [gene_id])
        ko_growth = model.optimize().objective_value
    pct_change = ((wt_growth - ko_growth) / wt_growth * 100) if wt_growth > 0 else 0
    return wt_growth, ko_growth, pct_change

# -----------------------------
# 6. Run knockouts on all genes
# -----------------------------
results = []
for gene in model.genes:
    wt, ko, pct = simulate_knockout(model, gene.id)
    results.append({"gene": gene.id, "WT_growth": wt, "KO_growth": ko, "Percent_change": pct})

# -----------------------------
# 7. Save to CSV
# -----------------------------
df = pd.DataFrame(results)
df.to_csv("knockout_results.csv", index=False)
print(f"Knockout results saved as 'knockout_results.csv' with {len(df)} genes.")

No objective coefficients in model. Unclear what should be optimized


Model loaded with 1044 reactions, 928 metabolites, 816 genes
All exchanges opened.


AttributeError: 'Objective' object has no attribute 'keys'

In [9]:
import cobra
import pandas as pd
import os

# --- CONFIGURE FILE PATHS ---
model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"
results_dir = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results"
output_file = os.path.join(results_dir, "knockout_results.csv")

# --- LOAD THE GEM ---
model = cobra.io.read_sbml_model(model_path)

# --- CHECK OR SET BIOMASS ---
if model.objective is None or "biomass" not in str(model.objective.expression).lower():
    print("No biomass reaction detected — please verify model reactions.")
else:
    print(f"Objective set to: {model.objective.expression}")

# --- CALCULATE WILD-TYPE GROWTH ---
wt_solution = model.optimize()
wt_growth = wt_solution.objective_value
print(f"Wild-type growth rate: {wt_growth:.4f}")

# --- BATCH GENE KNOCKOUTS ---
results = []
for gene in model.genes:
    with model:
        gene.knock_out()
        sol = model.optimize()
        ko_growth = sol.objective_value
        change = ((ko_growth - wt_growth) / wt_growth * 100) if wt_growth > 0 else 0
        results.append({
            "gene": gene.id,
            "WT_growth": wt_growth,
            "KO_growth": ko_growth,
            "Percent_change": change
        })

# --- SAVE RESULTS ---
df = pd.DataFrame(results)
df.to_csv(output_file, index=False)
print(f"Knockout results saved to: {output_file}")

No objective coefficients in model. Unclear what should be optimized


No biomass reaction detected — please verify model reactions.
Wild-type growth rate: 0.0000
Knockout results saved to: C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\knockout_results.csv


In [10]:
import cobra
import os
import pandas as pd

# Load the model
model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"
model = cobra.io.read_sbml_model(model_path)

# Step 1: Search for existing biomass-like reactions
biomass_candidates = [r for r in model.reactions if 'biomass' in r.id.lower() or 'growth' in r.id.lower()]

if biomass_candidates:
    biomass_rxn = biomass_candidates[0]
    print(f"Using existing biomass reaction: {biomass_rxn.id}")
else:
    # Step 2: Add a simple placeholder biomass reaction
    print("No biomass reaction found. Adding placeholder biomass reaction...")
    biomass_rxn = cobra.Reaction("BIOMASS_placeholder")
    biomass_rxn.name = "Placeholder Biomass Reaction"
    biomass_rxn.lower_bound = 0
    biomass_rxn.upper_bound = 1000
    # Use some abundant metabolites so the reaction can carry flux
    mets = [model.metabolites.get_by_id(m) for m in model.metabolites.list_attr("id")[:3]]
    biomass_rxn.add_metabolites({m: -1.0 for m in mets})  # Consume metabolites
    biomass_rxn.add_metabolites({model.metabolites.get_by_id(model.metabolites.list_attr("id")[0]): 1.0})  # Produce biomass
    model.add_reactions([biomass_rxn])

# Step 3: Set as objective
model.objective = biomass_rxn.id

# Step 4: Test wild-type growth
solution = model.optimize()
print(f"Wild-type growth rate: {solution.objective_value:.4f}")

# Step 5: Automate knockouts if growth > 0
results_dir = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results"
os.makedirs(results_dir, exist_ok=True)
results_path = os.path.join(results_dir, "knockout_results.csv")

if solution.objective_value > 0:
    knockout_results = []
    for gene in model.genes:
        with model:
            gene.knock_out()
            ko_solution = model.optimize()
            knockout_results.append({
                "gene": gene.id,
                "WT_growth": solution.objective_value,
                "KO_growth": ko_solution.objective_value,
                "percent_change": ((ko_solution.objective_value - solution.objective_value) / solution.objective_value) * 100
            })

    df = pd.DataFrame(knockout_results)
    df.to_csv(results_path, index=False)
    print(f"Knockout results saved to: {results_path}")
else:
    print("Growth is still zero — need a better biomass reaction definition.")

No objective coefficients in model. Unclear what should be optimized


Using existing biomass reaction: BIOMASS_Ec_SynAuto_1
Wild-type growth rate: 0.0765


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\DBAMK\\Documents\\ProkaryPredict\\CottonwoodIncubator\\ProkaryPredict-McCrory\\results\\knockout_results.csv'

In [11]:
import cobra
import pandas as pd
import os
import time

# ----------------- CONFIGURATION -----------------
model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"
results_dir = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results"
os.makedirs(results_dir, exist_ok=True)

# ----------------- LOAD MODEL -----------------
try:
    model = cobra.io.read_sbml_model(model_path)
    print(f"Model loaded successfully with {len(model.reactions)} reactions, {len(model.metabolites)} metabolites, and {len(model.genes)} genes.")
except Exception as e:
    raise RuntimeError(f"Error loading model: {e}")

# ----------------- BIOMASS SETUP -----------------
# Find biomass reaction
biomass_candidates = [rxn for rxn in model.reactions if "biomass" in rxn.id.lower()]
if not biomass_candidates:
    raise RuntimeError("No biomass reaction detected — you must define one first.")
else:
    biomass_rxn = biomass_candidates[0]
    model.objective = biomass_rxn.id
    print(f"Using biomass reaction: {biomass_rxn.id}")

# ----------------- WILD-TYPE GROWTH -----------------
solution = model.optimize()
if solution.status != "optimal":
    raise RuntimeError("Model failed to optimize. Check biomass reaction or constraints.")

wt_growth = solution.objective_value
print(f"Wild-type growth rate: {wt_growth:.4f}")

# ----------------- BATCH KNOCKOUTS -----------------
knockout_results = []
for gene in model.genes:
    with model:
        gene.knock_out()
        ko_solution = model.optimize()
        ko_growth = ko_solution.objective_value if ko_solution.status == "optimal" else 0.0
        change = ((ko_growth - wt_growth) / wt_growth * 100) if wt_growth > 0 else 0.0
        knockout_results.append({
            "gene": gene.id,
            "WT_growth": wt_growth,
            "KO_growth": ko_growth,
            "percent_change": change
        })

# ----------------- SAVE RESULTS -----------------
timestamp = time.strftime("%Y%m%d_%H%M%S")
results_path = os.path.join(results_dir, f"knockout_results_{timestamp}.csv")
df = pd.DataFrame(knockout_results)
df.to_csv(results_path, index=False, encoding='utf-8-sig')
print(f"Knockout results saved to: {results_path}")


No objective coefficients in model. Unclear what should be optimized


Model loaded successfully with 1044 reactions, 928 metabolites, and 816 genes.
Using biomass reaction: BIOMASS_Ec_SynAuto_1
Wild-type growth rate: 0.0765
Knockout results saved to: C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\knockout_results_20250928_203641.csv
